# Raw feature extraction script - from pcap files

In [1]:
import datetime
today = datetime.datetime.now()
last_updated = "Last updated on: " + today.strftime("%a, %m-%d-%Y")
print(last_updated)

Last updated on: Mon, 04-30-2018


In [2]:
import os, sys
import re

In [3]:
def split_pcaps(pcap_filename):
    """Read the pcaps, split them into chunks of smaller-sized pcaps"""
    
    pcap_chunks = re.findall('(.+).pcap', pcap_filename)[0] + '_mb.pcap' # first element of the list returned by findall()
    print(pcap_chunks)
    
    os.system("tcpdump -r " + pcap_filename + " -w " + pcap_chunks + " -C 50") # auto-creates smaller chunks
    small_pcaps = "ls -t " + pcap_chunks + "* > little_pcaps.txt" # enlists all the pcap chunks in a text file IN ORDER!!
    print(small_pcaps)    
    
    os.system(small_pcaps) # in the shell
    
    littlecaps = 'little_pcaps.txt'
    with open(littlecaps, 'r') as pcaplst:
        if len(pcaplst.readlines()) >= 1:
            return(littlecaps) # returns just the filename
    # if it failed to split pcaps
    return(0)

In [4]:
def extract_features_csv(pcap_filename):
    """Reads all the smaller-sized chunks and extracts all the necessary raw features into a csv file"""
    
    pcaplst = split_pcaps(pcap_filename) # first check if descriptor == 0
    
    if pcaplst == 0:
        print("Could not extract features -- pcaps not found")
        return(False)
    
    pcaplst = open(split_pcaps(pcap_filename), 'r') # file must be opened
    
    pcaplst = pcaplst.read().splitlines() # ignores the ending '\n' character
    n_chunks = len(pcaplst)
    print(n_chunks)
    
    if n_chunks != 0:
        fcsv = re.findall('(.+).pcap', pcap_filename)[0] + '.csv'  # creates a ISCX_Botnet_Testing.csv
        print(fcsv)
        csv_traffic = open(fcsv, 'w+') # close file at the end!
        for pcap in pcaplst[::-1]:
            
            '''Must install tshark for this to work.
            ~$ sudo apt-get install tshark'''
            
            # extract features and convert to csv
            os.system("tshark -nr " + pcap + " -T fields -e frame.number -t e -e _ws.col.Time -e ip.src -e ip.dst -e ip.proto -e ip.len -e ip.dsfield -e dns.count.answers -e tcp.dstport -e tcp.srcport -e frame.protocols -e ip.flags -e udp.length -e tcp.len -e tcp.seq -e tcp.nxtseq -e tcp.flags -e ip.geoip.dst_country -e ip.geoip.src_country -e ip.geoip.dst_lon -e ip.geoip.dst_lat -e ip.geoip.src_lon -e ip.geoip.src_lat -e ip.ttl -E header=y -E separator=, -E quote=d -E occurrence=f >> " + fcsv)
        csv_traffic.close()
        return(fcsv)
    else:
        return('Error -- no smaller-sized pcaps created')

In [5]:
os.getcwd()

'/home/aparna/Downloads'

In [12]:
# calling the functions on the pcap files
print(extract_features_csv('ISCX_Botnet_Testing.pcap'))

ISCX_Botnet_Testing_mb.pcap
ls -t ISCX_Botnet_Testing_mb.pcap* > little_pcaps.txt
44
ISCX_Botnet_Testing.csv
ISCX_Botnet_Testing.csv
